## Notes
### 02/09/18 [12:00]
Here are some notes after "finishing training".  
To kick things off, each epoch took aproximately 2h20min to train, in my personal Ubuntu. It isn't the greatest of machines, but I was able to get some very impressive results after 5 epochs:  
_Test loss: 1.4660126214812637  
Test accuracy: 0.8552342583948307_  
  
The lengthy duration of each epoch is the reason for why I didn't continue training for more epochs. And also the fact that python got to the point of consuming 1GB of RAM, making my computer downright unusable.  
Given that, the model results were saved after those 5 epochs.  
One of the reasons for such a long training time is that using 2d convolution makes it so that the model has way more trainable parameters, and I mean it:  
_Total params: 9,770,188  
Trainable params: 9,770,188  
Non-trainable params: 0_  
  
Also, something that I found during my research is this contrib feature: https://www.tensorflow.org/api_docs/python/tf/contrib/data/sliding_window_batch  
I'm not entirely sure of how to use it, but it might prove itself very useful if I ever need to train a network on some fixed data, so the network itself should do the sliding window data preparation.  
  
That being said, I'm happy with the obtained results. And as a last note, it's noticeable that the model began calibrating less and less after time, and I believe that is due to the optimizer used, the _Adadelta_, which has a adaptive momentum implementation. I speculate that using a SGD, this model could have reached accuracy close to the 90%-95% range within the same 5 epochs.  
  
### 01/09/18 [14:00 - 14:30]
As I mentioned in the last note, I tried implementing a 2d Convolution over the word Embeddings, almost as if they were images.  
To do so, I added a Reshape layer between the Embedding_1 and Conv1d_1, in which the final shape was (batch_size, window_size, embedding_dimensionality, 1). After that, I changed the 1d convolutions to 2d, changing the kernel_size from 3 to (3,3). With that, I can now explain the the addition of an apparently useless new dimensionality to the input vector after Reshape, and I can confirm that it is indeed useless. I mean, kinda. What it does is that conv2d requires a 4d input, with one of the dimensionalities being number of channels. So all that Reshape_1 does is that it includes a new "null" dimension that is treated as the number of channels, so that we can do spatial convolution on the Embeddings. 
And the results appear to be clear from the very start. I've only ran half an epoch so far, but accuracy results appear promising.  

### 12/08/18 [07:45 - 13:20]
So yeah, I might have done a mistake or two in the last implementation.... Between them, not realising that the input to Conv1D has a lot of channels and that I ~~could~~ should use a lot of filters.  
But that's all in the past, now that this little boy is up and learning and growing in a healthy manner.  
Now the first Conv1D uses 5 filters and the second one, 3. I could perhaps increase this someday.  
Not only that, but now it also saves it weights during training (I had done this in another implementation, but I forgot to commit that one, so now it's in this one).  
And now on to some notes for my future self, I was asing myself if it would be possible to get the 16 dimensional vector produced by Embedding and use Conv2D on it. It seems to be possible with the aid of keras.layers.Reshape. The idea would be to simply add a single dimension to the end of Embedding, so it would become 4D, allowing it to be used as input to Conv2D. I might do that in a separate file though, so that I'm able to compare both results.
Another possible improvement might be to shuffle the data. I didn't do so because there data order is relevant in this data set, but who knows what might happen?  
Also, here goes a complete log of the changes, in chronological order:
- Changed the parameter filter in the first Conv1D to 5
- Changed the parameter filter in the second Conv2D to 3
- Changed metrics to 'acc', as there is no implementation of sparse_categorical_accuracy in tf.keras
- Added saving cell, with model.save_weights
- Added ModelCheckpoint callback, saving only weights
- Added model saving to saving cell
- Changed the ModelCheckpoint parameter save_only_weights to False, as to resume training in later ocasions
- Also, increased periods in ModelCheckpoint
- Changed the checkpoint path to include data and time of checkpoint
- Gave names to the layers
- Added loading model cell  

On a side note, loss just went nuts


### Some time in the past:
I have yet to conclude training the model, but I was to excited to share it finally working after a long time.  
On a side note, however, it does seem to overfit before the end of even the first epoch, but we need to conclude a whole training session before jumping to any conclusions


# Word Prediction using TDNN implemented in tf.keras
This notebook is an attempt at implementing a Time Delay Neural Network for word prediction in the ptb dataset

In [2]:
# Imports
from __future__ import print_function
import tensorboard as tb
from tensorflow import keras
import reader
import numpy as np
import os
from time import strftime, gmtime
import pathlib
from pathlib import PosixPath

In [3]:
# Constants
window_size = 20 # defines the past lookup for determining the following word
path = "data/simple-examples/data"
checkpoint_path = "training/conv2d/cp-{epoch:04d}_" + strftime("%H-%M_%d-%m-%Y", gmtime()) + ".ckpt"
checkpoint_dir = os.path.dirname

The following cell obtain the data using the reader.py file

In [4]:
train_data, valid_data, test_data, vocab_size, word_to_id = reader.ptb_raw_data(path)
x_train = train_data[:-1]
x_train = [np.asarray(x_train[i:i+window_size]) for i in range(len(x_train)-window_size)]
x_train = np.asarray(x_train)
y_train = np.asarray(train_data[1:-window_size])
#y_train = keras.preprocessing.text.one_hot(y_train, vocab_size)
x_valid = valid_data[:-1]
x_valid = [np.asarray(x_valid[i:i+window_size]) for i in range(len(x_valid)-window_size)]
x_valid = np.asarray(x_valid)
y_valid = valid_data[1:-window_size]
y_valid = np.asarray(y_valid)
x_test = test_data[:-1]
x_test = [np.asarray(x_test[i:i+window_size]) for i in range(len(x_test)-window_size)]
x_test = np.asarray(x_test)
y_test = test_data[1:-window_size]
y_test = np.asarray(y_test)
id_to_word = {value: key for (key, value) in word_to_id.items()}

Following next, we have an auxiliary function which decodes the ids and give us the original sentences

In [5]:
def decode_text(text):
    return ' '.join([id_to_word.get(i, '?') for i in text])

In [6]:
decode_text (x_train[40])

'<eos> mr. <unk> is chairman of <unk> n.v. the dutch publishing group <eos> rudolph <unk> N years old and former'

In [7]:
print(x_train.shape)
print(y_train.shape)

(929568, 20)
(929568,)


As we can see, each index of the input has a length of 20 words, as defined in _window-length_
<br>
With our data already processed, we can finally create our model.

In [8]:
# First time instantiation
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_length = window_size, name = "Embedding_1"))
#model.add(keras.layers.Flatten())
model.add(keras.layers.Reshape((20,16,1), name = "Reshape_1"))
model.add(keras.layers.Conv2D(filters = 5, kernel_size = (3,3), padding = "same", activation = keras.activations.tanh, name = "Conv2D_1"))
model.add(keras.layers.Dropout(0.2, name = "Droupout_1"))
model.add(keras.layers.Conv2D(filters = 3, kernel_size = 3, padding = "same", activation = keras.activations.tanh, name = "Conv2D_2"))
model.add(keras.layers.Dropout(0.25, name = "Dropout_2"))
model.add(keras.layers.Flatten(name = "Flatten_1"))
model.add(keras.layers.Dense(vocab_size, activation = keras.activations.softmax, name = "Dense_1"))

In [18]:
# Model loading - loads the most recent model recorded manually. Feel free to change
# Sort the checkpoints by modification time.
checkpoints = pathlib.Path("./models").glob("*")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints
model = keras.models.load_model(latest)

In [9]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_1 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
Reshape_1 (Reshape)          (None, 20, 16, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 20, 16, 5)         50        
_________________________________________________________________
Droupout_1 (Dropout)         (None, 20, 16, 5)         0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 20, 16, 3)         138       
_________________________________________________________________
Dropout_2 (Dropout)          (None, 20, 16, 3)         0         
_________________________________________________________________
Flatten_1 (Flatten)          (None, 960)               0         
__________

In [10]:
model.compile(
    loss = keras.losses.sparse_categorical_crossentropy,
    optimizer = keras.optimizers.Adadelta(),
    metrics = ['acc'] #keras.metrics.categorical_accuracy] # remember to later change to sparse_categorical_accuracy (this is the cause for strange eval)
)

In [11]:
# Checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose = 1, save_weights_only = False, period = 3) # Also, later change to save_weights_only = false, and perhaps increase period to 3 or 4. This will allow us to later resume training from where we left

In [15]:
model.fit(x_train, y_train,
          epochs = 12,
          verbose = 1,
          validation_data = (x_valid, y_valid),
          shuffle = False,
          callbacks = [cp_callback]
)

Train on 929568 samples, validate on 73739 samples
Epoch 1/12
929568/929568 [==============================] - 7905s 9ms/step - loss: 1.9855 - acc: 0.7735 - val_loss: 1.7232 - val_acc: 0.8218
Epoch 2/12
929568/929568 [==============================] - 7865s 8ms/step - loss: 1.8789 - acc: 0.7881 - val_loss: 1.6392 - val_acc: 0.8333
Epoch 3/12
929568/929568 [==============================] - 7881s 8ms/step - loss: 1.8093 - acc: 0.7969 - val_loss: 1.5854 - val_acc: 0.8404

Epoch 00003: saving model to training/conv2d/cp-0003_20-54_01-09-2018.ckpt
Epoch 4/12
929568/929568 [==============================] - 7894s 8ms/step - loss: 1.7597 - acc: 0.8030 - val_loss: 1.5460 - val_acc: 0.8456
Epoch 5/12
929568/929568 [==============================] - 8130s 9ms/step - loss: 1.7246 - acc: 0.8075 - val_loss: 1.5162 - val_acc: 0.8492
Epoch 6/12
 22048/929568 [..............................] - ETA: 2:12:19 - loss: 1.8817 - acc: 0.7875

KeyboardInterrupt: 

In [17]:
# I expect to be able to run this someday
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.4660126214812637
Test accuracy: 0.8552342583948307


In [16]:
model.save_weights("./model_weights/conv2d_weights_" + strftime("%H-%M_%d-%m-%Y", gmtime()))
model.save("./models/conv2d_model_" + strftime("%H-%M_%d-%m-%Y", gmtime())) # This allows us to resume training, since Adadelta has adaptive parameters